# Synchrotron Data Visualisation Tool

This notebook helps you visualise your synchrotron data.

## Setup
First, we'll import the necessary packages. Run each cell by pressing Shift+Enter.

In [7]:
import os
import glob
import h5py
import yaml
import httomo
import numpy as np
from data_loader import NXSDataLoader
import sys
sys.path.append("./stackview")
import stackview
sys.path.append("./yaml_editor") 
from yaml_editor_class import YAML_Editor

import subprocess
import shutil

## Load Your Data
In the cell below, update the `visit_number` and `year` variables with your details.

In [2]:
visit_number = 'cm40627-1'
year = '2025'

browser = NXSDataLoader(visit_number, year)

## Visualise Projections
Once your data is loaded run the cell below to launch the interactive viewer. You can:
- Navigate through projections using the Slice slider
- Adjust contrast using the Max and Min sliders

In [3]:
stackview.viewer(browser.current_data, continuous_update=True, zoom_factor=1.2)

In [4]:
editor = YAML_Editor("/dls/k11/data/" + year + "/" + visit_number + "/xml/tomography/configurations/HTTomo")
editor.Editor()

In [5]:
print(browser.filepath)
print(editor.save_path)
print(editor.current_data)

/dls/k11/data/2025/cm40627-1/nexus/k11-53999.nxs
/dls/k11/data/2025/cm40627-1/xml/tomography/configurations/HTTomo/DIAD_VO_RS_FBP_edit.yaml
[{'method': 'standard_tomo', 'module_path': 'httomo.data.hdf.loaders', 'parameters': {'data_path': '/entry/imaging/data', 'image_key_path': '/entry/instrument/imaging/image_key', 'rotation_angles': {'data_path': '/entry/imaging_sum/gts_cs_theta'}}}, {'id': 'centering', 'method': 'find_center_vo', 'module_path': 'httomolibgpu.recon.rotation', 'parameters': {'drop': 20, 'ind': 'mid', 'ratio': 0.5, 'smax': 50, 'smin': -50, 'srad': 6, 'step': 0.25}, 'side_outputs': {'cor': 'centre_of_rotation'}}, {'method': 'remove_outlier', 'module_path': 'httomolibgpu.misc.corr', 'parameters': {'dif': 0.1, 'kernel_size': 3}}, {'method': 'normalize', 'module_path': 'httomolibgpu.prep.normalize', 'parameters': {'cutoff': 10.0, 'minus_log': True, 'nonnegativity': False, 'remove_nans': False}}, {'method': 'remove_stripe_based_sorting', 'module_path': 'httomolibgpu.prep.s

In [14]:
output_path = f"/dls/k11/data/{year}/{visit_number}/tmp/live_recon"
create_folder = f"mkdir {output_path}"

httomo_load = 'module load openmpi && module load mamba && module load httomo && module list'
httomo_reco = f"python -m httomo run {browser.filepath} {editor.save_path} {output_path}"

try:
    shutil.rmtree(output_path)
except:
    pass
result = subprocess.run(f'{create_folder}', shell=True, capture_output=True, text=True)
print("Destination Folder Output:", result.stdout)
print("Destination Folder Error:", result.stderr)

result = subprocess.run(f'{httomo_load} && {httomo_reco}', shell=True, capture_output=True, text=True)
print("HtTomo Output:", result.stdout)
print("HtTomo Error:", result.stderr)

print('END OF RECONSTRUCTION')

Destination Folder Output: 
Destination Folder Error: 
HtTomo Output: 	 OpenMPI (4.1.6) environment set up (x86_64 bit version)

Mamba will cache packages in /scratch/30day_tmp/cxh52226/.conda/pkgs

		  *********************************************
		   *    Loading HTTomo version 2.3 *
		  *********************************************


Loading httomo/2.3
  Loading requirement: openmpi/4.1.5
Currently Loaded Modulefiles:
 1) use.own           5) global/directories      9) openmpi/4.1.5        
 2) controls-tools    6) python/3.10(default)   10) httomo/2.3(default)  
 3) controls          7) openmpi/4.1.6__cuda12  
 4) epics/3.14.12.7   8) mamba/2.0.5(default)   
Failed to import module in /dls_sw/apps/httomo/2.3/lib/python3.12/site-packages/httomolibgpu/cupywrapper.py with error: 
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBR

In [23]:
processed_data = None
for root, dirs, files in os.walk(output_path):
    for file in files:
        if file.endswith(".log"):
            processed_data = os.path.join(root, file)
            pass

In [26]:
output_path = "/dls/k11/data/2025/cm40627-1/nexus/k11-53999.nxs"
current_data = None
with h5py.File(output_path, 'r') as f:
    current_data = np.asarray(f['entry/imaging/data'][:], dtype=np.uint16) ## update entry name !!!!!!
stackview.viewer(current_data, continuous_update=True, zoom_factor=1.2)